
# 🚀 YOLOv11: Object Detection Demo (Install → mAP/IoU → Fine‑tune → License)

This notebook introduces **YOLOv11** using the Ultralytics package. You'll:
- Install and verify the `ultralytics` package (YOLO CLI + Python API).
- Run a quick **inference** demo.
- Understand **IoU** and **mAP** with small, concrete examples.
- **Fine‑tune** a YOLOv11 model on a toy dataset and evaluate.
- Review **licensing** considerations for research vs. commercial use.

> **Note:** Cells that download models/data require internet access in your environment.


## 0) Install Ultralytics

In [1]:
# !pip uninstall opencv-python -y
# !pip install opencv-python

In [2]:

# If not installed, uncomment the next line
# !pip install -U ultralytics
import sys, torch
print('Python:', sys.version)
print('PyTorch:', torch.__version__)

try:
    import ultralytics
    from ultralytics import YOLO
    print('Ultralytics:', ultralytics.__version__)
except Exception as e:
    print("Ultralytics not available yet. Run pip install above.\n", e)


Python: 3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]
PyTorch: 2.5.0+cu124
Ultralytics: 8.3.205


## 1) Quick Start: Pretrained Inference

In [3]:

from ultralytics import YOLO

MODEL = 'yolo11n.pt'  # or 'yolo11s.pt'
model = YOLO(MODEL)

# Predict on a sample image (downloads on first run)
# pred = model.predict(source='https://ultralytics.com/images/bus.jpg', save=True, imgsz=640)
pred = model.predict(source='./bus.jpg', save=True, imgsz=640)
pred[0].boxes[:5] if len(pred) else print("No result")



image 1/1 /home/kperkins411/471_code/week7/bus.jpg: 640x480 4 persons, 1 bus, 36.4ms
Speed: 6.6ms preprocess, 36.4ms inference, 62.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /home/kperkins411/471_code/week7/runs/detect/predict5


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([5., 0., 0., 0., 0.], device='cuda:0')
conf: tensor([0.9402, 0.8881, 0.8782, 0.8558, 0.6219], device='cuda:0')
data: tensor([[3.8074e+00, 2.2936e+02, 7.9621e+02, 7.2842e+02, 9.4019e-01, 5.0000e+00],
        [6.7102e+02, 3.9485e+02, 8.0981e+02, 8.7871e+02, 8.8812e-01, 0.0000e+00],
        [4.7413e+01, 3.9957e+02, 2.3930e+02, 9.0420e+02, 8.7822e-01, 0.0000e+00],
        [2.2306e+02, 4.0868e+02, 3.4447e+02, 8.6044e+02, 8.5577e-01, 0.0000e+00],
        [2.1391e-02, 5.5607e+02, 6.8883e+01, 8.7236e+02, 6.2188e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 810)
shape: torch.Size([5, 6])
xywh: tensor([[400.0092, 478.8904, 792.4037, 499.0573],
        [740.4153, 636.7804, 138.7878, 483.8646],
        [143.3546, 651.8820, 191.8829, 504.6299],
        [283.7636, 634.5623, 121.4106, 451.7573],
        [ 34.4523, 714.2173,  68.8618, 316.2908]], device='cuda:0')
xywhn: tensor([[0.4938, 0.4434, 0.97

## 2) Metrics: IoU and mAP (What they mean)


**Intersection over Union (IoU)** measures how well a predicted box overlaps a ground‑truth box:
\[ \text{IoU} = \frac{\text{Area}(\text{Pred} \cap \text{GT})}{\text{Area}(\text{Pred} \cup \text{GT})} \]
- IoU ∈ [0, 1]. Higher is better; an IoU of 0.5 is common as a positive match threshold.

**Average Precision (AP)** summarizes the **precision–recall** curve for one class at a given IoU threshold.

**mAP** is the **mean AP across classes**, sometimes averaged across multiple IoU thresholds:
- **mAP@0.5** (VOC style)
- **mAP@0.5:0.95** (COCO style)


In [4]:

# Simple IoU demo for axis-aligned boxes (x1, y1, x2, y2)
def box_iou(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1, inter_y1 = max(ax1, bx1), max(ay1, by1)
    inter_x2, inter_y2 = min(ax2, bx2), min(ay2, by2)
    inter_w = max(0.0, inter_x2 - inter_x1)
    inter_h = max(0.0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    union = area_a + area_b - inter
    return inter / union if union > 0 else 0.0

gt = (10, 10, 50, 50)
preds = [(12, 12, 48, 48), (40, 40, 80, 80), (60, 60, 90, 90)]
[round(box_iou(gt, p), 3) for p in preds]


[0.81, 0.032, 0.0]

## 3) Evaluate on a small dataset (COCO128)

In [ ]:

from ultralytics import YOLO
model = YOLO('yolo11n.pt')
metrics = model.val(data='coco128.yaml', imgsz=640)  # downloads tiny dataset
metrics.box.map, metrics.box.map50, metrics.box.maps[:5]


## 4) Fine‑tuning YOLOv11 (custom data)


### YOLO data.yaml
```yaml
path: /path/to/dataset
train: images/train
val:   images/val
names:
  0: classA
  1: classB
```

### CLI
```bash
yolo detect train model=yolo11s.pt data=/path/to/data.yaml imgsz=640 epochs=50 batch=16 amp=True
```

### Python API
```python
from ultralytics import YOLO
model = YOLO('yolo11s.pt')
results = model.train(data='/path/to/data.yaml', imgsz=640, epochs=50, batch=16, optimizer='AdamW', amp=True)
```


## 5) Licensing notes


Ultralytics YOLOv11 is distributed under **AGPL‑3.0** by default. For **commercial distribution** (shipping products or services),
either comply with AGPL‑3.0 obligations or obtain a **commercial license** from Ultralytics.

Always check the latest terms on Ultralytics’ website before deployment.
